<a href="https://colab.research.google.com/github/hussain0048/Projects-/blob/master/Real_Time_Face_Mask_Detector_(Not_Completed)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1-Introducion**

During pandemic COVID-19, WHO has made wearing masks compulsory to protect against this deadly virus. In this tutorial we will develop a machine learning project – Real-time Face Mask Detector with Python.

We will build a real-time system to detect whether the person on the webcam is wearing a mask or not. We will train the face mask detector model using Keras and OpenCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [17]:
!unzip -uq "/content/drive/My Drive/Datasets/Real time Mask Detection /train.zip" -d "/content/drive/My Drive/Datasets/Real time Mask Detection /train"


In [18]:
!unzip -uq "/content/drive/My Drive/Datasets/Real time Mask Detection /test.zip" -d "/content/drive/My Drive/Datasets/Real time Mask Detection /test"


#**2- Dataset**
The dataset we are working on consists of 1376 images with 690 images containing images of people wearing masks and 686 images with people without masks.

#**2-Import libraries and modules**

In [20]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

#**3- Build the neural network**

This convolution network consists of two pairs of Conv and MaxPool layers to extract features from the dataset. Which is then followed by a Flatten and Dropout layer to convert the data in 1D and ensure overfitting.

And then two Dense layers for classification.

In [23]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

In [24]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

##**4-Image Data Generation/Augmentation**

In [22]:
TRAINING_DIR = "/content/drive/My Drive/Datasets/Real time Mask Detection /train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/content/drive/My Drive/Datasets/Real time Mask Detection /test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 1 classes.
Found 194 images belonging to 1 classes.


##5-**Initialize a callback checkpoint**
Initialize a callback checkpoint to keep saving best model after each epoch while training:

In [25]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

## **6-Train the model**

In [26]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
132/132 [==============================] - ETA: 0s - loss: 0.7929 - acc: 0.9772WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model2-001.model/assets
132/132 [==============================] - 156s 1s/step - loss: 0.7929 - acc: 0.9772 - val_loss: 0.6931 - val_acc: 1.0000
Epoch 2/10
132/132 [==============================] - 149s 1s/step - loss: 0.6931 - acc: 1.0000 - val_loss: 0.6931 - val_acc: 1.0000
Epoch 3/10
132/132 [============================

#**7- Test the model** 

Now we will test the results of face mask detector model using OpenCV.

Make a python file “test.py” and paste the below script.

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("./model-010.h5")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier('/home/user_name/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()

# **References** 
[[1] Real-Time Face Mask Detector with Python](https://https://data-flair.training/blogs/face-mask-detection-with-python/)